# Lab 4: Coffee Lovers Unite

## Scrape Location counts

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

### 1. Scrape data from Menuism website

In [2]:
#Set up lists:

us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

url_list = ['starbucks-coffee-39564', 'dunkin-donuts-181624', 'peets-coffee-tea-84051'
            'tim-hortons-190025', 'panera-bread-4258', 'caribou-coffee-164861', 'au-bon-pain-69342',
            'the-coffee-bean-tea-leaf-165988', 'mcdonalds-21019']

base_url = "https://www.menuism.com/restaurant-locations/"

In [3]:
rows1 = []

for store_url in url_list:
    full_url = base_url + store_url

    # Send an HTTP GET request to the URL
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, "html.parser")

    chains = soup.find_all("ul", attrs={"class": "list-unstyled-links"})

    # Initialize an empty list
    location_count_list = []

    # Iterate over all course items
    for chain in chains:
        location_tags = chain.find_all("a")
        for location_tag in location_tags:
            location_count = location_tag.text
            location_count_list.append(location_count)

    # Split the location data to separate "Location" and "Store Count"
    location_split = [location.split("(") for location in location_count_list]

    # Append this data to rows1
    for location, count in location_split:
      for states in us_states:
        if states in location:
          rows1.append({
            "State": location.strip(),
            "Store Count": count.strip(")")
        })

#Create a DataFrame from the list of dictionaries
rows1 = pd.DataFrame(rows1)

### 2. State abbreviations

In [4]:
def stateabb(state):
    states_dict = {
        "Alabama": "AL",
        "Alaska": "AK",
        "Arizona": "AZ",
        "Arkansas": "AR",
        "California": "CA",
        "Colorado": "CO",
        "Connecticut": "CT",
        "Delaware": "DE",
        "Florida": "FL",
        "Georgia": "GA",
        "Hawaii": "HI",
        "Idaho": "ID",
        "Illinois": "IL",
        "Indiana": "IN",
        "Iowa": "IA",
        "Kansas": "KS",
        "Kentucky": "KY",
        "Louisiana": "LA",
        "Maine": "ME",
        "Maryland": "MD",
        "Massachusetts": "MA",
        "Michigan": "MI",
        "Minnesota": "MN",
        "Mississippi": "MS",
        "Missouri": "MO",
        "Montana": "MT",
        "Nebraska": "NE",
        "Nevada": "NV",
        "New Hampshire": "NH",
        "New Jersey": "NJ",
        "New Mexico": "NM",
        "New York": "NY",
        "North Carolina": "NC",
        "North Dakota": "ND",
        "Ohio": "OH",
        "Oklahoma": "OK",
        "Oregon": "OR",
        "Pennsylvania": "PA",
        "Rhode Island": "RI",
        "South Carolina": "SC",
        "South Dakota": "SD",
        "Tennessee": "TN",
        "Texas": "TX",
        "Utah": "UT",
        "Vermont": "VT",
        "Virginia": "VA",
        "Washington": "WA",
        "West Virginia": "WV",
        "Wisconsin": "WI",
        "Wyoming": "WY"
    }
    
    if state in states_dict:
        return  states_dict[state]
    else:
        return None

    return states_dict[state]

In [5]:
stateabb("Georgia")

'GA'

   ### 3. Parse, merge, and tidy

In [6]:
# Tidying the data
rows1 = rows1.groupby("State", as_index=False).agg({"Location Count": "sum"})
rows1.columns = ["State Abbreviation", "Location Count"]
print(rows1)

KeyError: 'State'

### 4. Wikipedia scrape

In [7]:
pop_table = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")[0]

pop_list = ["State", "Census population, April 1, 2020 [1][2]"]
pop_table = pop_table[pop_list]
pop_table.head()

,State,"Census population, April 1, 2020 [1][2]"
0,California,39538223
1,Texas,29145505
2,Florida,21538187
3,New York,20201249
4,Pennsylvania,13002700


In [8]:
rows1 = pd.read_csv(r'C:\Users\ryanc\Desktop\GSB_544\df_lab4_scraped.csv')

# Display the first few rows of the DataFrame
rows1.head()

,State,Coffee Chain,State Abbreviation,Location Count,Population
0,Alaska,Starbucks,AK,24,733391
1,Alabama,Starbucks,AL,73,5024279
2,Arkansas,Starbucks,AR,33,3011524
3,Arizona,Starbucks,AZ,279,7151502
4,California,Starbucks,CA,2362,39538223


### 5. Add stock prices

In [9]:
def locate_stock(chain):
    stock_prices = {
        "Starbucks": 93.15,
        "Dunkin’ Donuts": 106.48,
        "Peet's Coffee & Tea": 8.00,
        "Tim Horton’s": 71.61,
        "Panera Bread": 314.93,
        "Caribou Coffee": 16.49,
        "Au Bon Pain": 6.25,
        "The Coffee Bean & Tea Leaf": 3.78
    }
    return stock_prices.get(chain, 260.15) 
rows1["Stock Price"] = rows1["Coffee Chain"].apply(locate_stock)
rows1

,State,Coffee Chain,State Abbreviation,Location Count,Population,Stock Price
0,Alaska,Starbucks,AK,24,733391,93.15
1,Alabama,Starbucks,AL,73,5024279,93.15
2,Arkansas,Starbucks,AR,33,3011524,93.15
3,Arizona,Starbucks,AZ,279,7151502,93.15
4,California,Starbucks,CA,2362,39538223,93.15
...,...,...,...,...,...,...
264,Vermont,McDonald's,VT,30,643077,260.15
265,Washington,McDonald's,WA,326,7705281,260.15
266,Wisconsin,McDonald's,WI,353,5893718,260.15
267,West Virginia,McDonald's,WV,107,1793716,260.15


### 6. Add regions

In [10]:
state_regions = {
    'Northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania'],
    'Midwest': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'],
    'South': ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia', 'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
    'West': ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
}

# Function to check the region
def check_region(state):
    for region, states in state_regions.items():
        if state in states:
            return region
    return None  # Return None if state not found

# Adding a new column 'Region'
rows1['Region'] = rows1['State'].apply(check_region)

# Displaying the DataFrame
rows1

,State,Coffee Chain,State Abbreviation,Location Count,Population,Stock Price,Region
0,Alaska,Starbucks,AK,24,733391,93.15,West
1,Alabama,Starbucks,AL,73,5024279,93.15,South
2,Arkansas,Starbucks,AR,33,3011524,93.15,South
3,Arizona,Starbucks,AZ,279,7151502,93.15,West
4,California,Starbucks,CA,2362,39538223,93.15,West
...,...,...,...,...,...,...,...
264,Vermont,McDonald's,VT,30,643077,260.15,Northeast
265,Washington,McDonald's,WA,326,7705281,260.15,West
266,Wisconsin,McDonald's,WI,353,5893718,260.15,Midwest
267,West Virginia,McDonald's,WV,107,1793716,260.15,South


### 7. Inferences

- McDonald's also has a strong presence in the South with 6,606 locations, indicating a potential preference or strategic placement in this region.
- Caribou Coffee is more prevalent in the Midwest, especially states like Minnesota.
- Tim Horton's has more outlets in the Midwest compared to other regions, suggesting a strategic focus on this region.
- Starbucks, despite being highly prevalent in the West, also has a significant number of stores in the Midwest and South, indicating a well-distributed presence across the US. Not a surprise, it's Starbucks...
- Comparing the number of stores to the state's population, having numerous stores in highly populated states like California might be more beneficial than having the same number in less populated states like Wyoming.


## Automate

### Convert your code for Exercises 1-3 above to a function that takes a single argument: the menuism URL. This function should

In [ ]:
def scrape_menuism(url):
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    state_names = [div.get_text() for div in soup.find_all('div', class_='state-name')]
    location_counts = [int(div.get_text()) for div in soup.find_all('div', class_='location-count')]
    
    company_name = url.split('/')[-2].replace('-coffee', '').replace('-locations', '').capitalize()
    
    df = pd.DataFrame({
        'State Abbreviation': [stateabb(state) for state in state_names],
        'Location Count': location_counts,
        'Company Name': [company_name] * len(state_names)
    })
    
    return df